In [1]:
# -*- coding:utf-8 -*-
import urllib.request
from urllib.parse import quote
import string

import json
import sys
import time
import pandas as pd   
import numpy as np
from urllib.error import HTTPError
#import codecs
#import os
#import urllib2

In [2]:
class LocaDiv(object):         #定义，分区类，输入一个分区，以字符串形式
    def __init__(self, loc):
        self.loc = loc
        
#    def ls_row(self):  分区函数，输入一个区，分成9个区
    def new_bounds(self):    
    
        lat_sw = float(self.loc.split(',')[0])        #按逗号，分割字符串loc_all，取第一个
        lat_ne = float(self.loc.split(',')[2])
        lng_sw = float(self.loc.split(',')[1])
        lng_ne = float(self.loc.split(',')[3])
  
        new_bounds = []
        #  col_row 将bounds的每一小块继续细分为3行3列，可以防止区域内的搜索数量上限400
        col_row = 3 
        #for lst in bounds:
        distance_lat = (lat_ne - lat_sw)/col_row
        distance_lon = (lng_ne - lng_sw)/col_row
        for i in range(col_row):
            for j in range(col_row):
                lst_temp = []
                lst_temp.append(lat_sw+distance_lat*i)
                lst_temp.append(lng_sw+distance_lon*j)
                lst_temp.append(lat_sw+distance_lat*(i+1))
                lst_temp.append(lng_sw+distance_lon*(j+1))
                new_bounds.append(lst_temp)
        
        return new_bounds

In [3]:
class BaiDuPOI(object):         # inpu:  itemy  POI类型，   loc 区域
    def __init__(self, loc):
        #self.itemy = itemy
        self.loc = loc

    def baidu_search(self):
        
        json_sel = []        #搜索结果list  每一个区域搜索 返回的结果，写入csv
        page_size = 20
        page_num = 0
        coord_type = 3
        np = True
        a=[]
        i = 0      # ak 索引
        while np == True:
            total = 0
            url="http://api.map.baidu.com/place/v2/search?ak="+str(ak[i])+"&output=json&query="+str(itemy[0])+ \
                "&page_size="+str(page_size)+"&page_num="+str(page_num)+"&coord_type="+ str(coord_type) + \
                "&bounds="+str(self.loc[0])+","+str(self.loc[1])+","+str(self.loc[2])+","+str(self.loc[3])
            
            #url = 'http://api.map.baidu.com/place/v2/search?query=' + self.itemy + '&bounds=' + self.loc + '&page_size=20&page_num=' + \
           #     str(pages) + '&output=json&ak=' + api_key
            
            url = quote(url, safe=string.printable)
            
            #请求url读取，创建网页对象
            
            try:
                jsonf = urllib.request.urlopen(url)
            except HTTPError:
                print("HTTPError，第:",i+1,"个ak发生网络错误",ak[i] )
                i += 1
                if i >= len(ak):
                    print("ak已用完，请明天再下载")
                    sys._exit(0)            #退出，引发一个可捕获异常
                else:
                    print("用第:", i+1,"个ak,继续下载", ak[i])
                    continue
                
            
            page_num=page_num+1
        
            #判断查询翻页进程
            jsonfile=jsonf.read()
            s=json.loads(jsonfile)
           # print(s)
            total=int(s["total"])
            #total=int(s.get("total"))
            a.append(total)
            
            # for url in self.urls():        #
            #json_obj = urllib2.urlopen(url)
            #data = json.load(json_obj)
            for item in s['results']:
                jname = item["name"]
                jlat = item["location"]["lat"]
                jlng = item["location"]["lng"]
                js_sel = jname + ',' + str(jlat) + ',' + str(jlng)
                json_sel.append(js_sel)
                               
            max_page=int(a[0]/page_size)+1
            #防止并发过高，百度地图要求并发小于120
            time.sleep(1) 
        
            if page_num>max_page:
                np=False
                page_num=0
                
        print("search complete")
        print("output: ", str(self.loc))
        print("total: ", str(a[0]))
        print(json_sel)
        print("")
                
        return json_sel,a[0]
                
        
    def write_poi(self, json_sel):
        #filename =[".\\POI\\a_stdout_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log']
        doc = open(".\\results.csv", 'a+')
        #doc.write(codecs.BOM_UTF8)
        #for ax in json_sel:
        #    doc.write(ax)
        #    doc.write('\n')
        for item in range(0,len(json_sel)):
            doc.write(json_sel[item])
            doc.write("\n")
        doc.close()       
        
        

In [4]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a+')

    def write(self, message):
        self.terminal.write(message)
        self.terminal.flush()
        self.log.write(message)
        self.log.flush()

    def flush(self):
        pass

filename1 =[".\\log\\a_stdout_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log']
#filename2 =[".\\log\\a_stderr_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log_file']
    
#sys.stdout = Logger(".\\a_stdout.log", sys.stdout)
#sys.stderr = Logger(".\\a_stderr.log_file", sys.stderr)             # redirect std err, if necessary
sys.stdout = Logger(filename1[0], sys.stdout)
#sys.stderr = Logger(filename2[0], sys.stderr)    

开始爬取数据，请稍等... 2019-07-10 08:41:34
[[40.02073567, 116.3088807, 40.074925113333336, 116.37735956666667], [40.02073567, 116.37735956666667, 40.074925113333336, 116.44583843333334], [40.02073567, 116.44583843333334, 40.074925113333336, 116.5143173], [40.074925113333336, 116.3088807, 40.129114556666664, 116.37735956666667], [40.074925113333336, 116.37735956666667, 40.129114556666664, 116.44583843333334], [40.074925113333336, 116.44583843333334, 40.129114556666664, 116.5143173], [40.129114556666664, 116.3088807, 40.183304, 116.37735956666667], [40.129114556666664, 116.37735956666667, 40.183304, 116.44583843333334], [40.129114556666664, 116.44583843333334, 40.183304, 116.5143173]]
现在爬取第 1 大区的第 1 区域
search complete
output:  [40.02073567, 116.3088807, 40.074925113333336, 116.37735956666667]
total:  350
['麦当劳(上地十街餐厅),40.059343,116.309', '肯德基(上地餐厅),40.035997,116.319883', '田老师红烧肉(上地六街店),40.048692,116.309925', '麦当劳(学清路大莲花餐厅),40.020768,116.359449', '麦当劳(清河餐厅),40.042098,116.342652', '肯德基(西三旗桥东餐厅),40.


现在爬取第 1 大区的第 2 区域
search complete
output:  [40.02073567, 116.37735956666667, 40.074925113333336, 116.44583843333334]
total:  233
['麦当劳(北苑路餐厅),40.037961,116.423813', '稻香村(天通苑店),40.071529,116.423498', '麦当劳(清苑路餐厅),40.05272,116.439734', '肯德基(北苑餐厅),40.039277,116.425981', '麦当劳(立水桥餐厅),40.063432,116.422041', '肯德基(立水桥餐厅),40.054421,116.413881', '麦当劳(北苑秋实路餐厅),40.048049,116.428175', '麦当劳(安立路二号餐厅),40.049854,116.417232', '汉堡王(北京龙德广场餐厅),40.065481,116.421847', '肯德基(立汤路餐厅),40.066619,116.421234', '嘉和一品(北苑店),40.040255,116.425639', '嘉和一品(立汤路店),40.063875,116.422515', '田老师红烧肉(中东路店),40.064009,116.415004', '周黑鸭(北苑店),40.039581,116.424888', '明洞欧巴韩式快餐,40.06745,116.416884', '赛百味(北苑路店),40.03793,116.423106', '和合谷(北苑店),40.046934,116.422339', '西部杨华拉面(宝盛里店),40.039951,116.378922', '沙县小吃(红军营店),40.038394,116.430813', '吉野家(华贸天地店),40.052591,116.439379', '螺阿宝柳州螺蛳粉,40.060105,116.418265', '田老师红烧肉(西一区店),40.074441,116.416625', '珊泉居黄焖鸡米饭(立水桥店),40.060033,116.417861', '好适口(天通东一区店),40.074197,116.442891', '好适口经典主食(富力桃园店),40.070172,

output:  [40.02073567, 116.44583843333334, 40.074925113333336, 116.5143173]
total:  73
['麦当劳(来广营西路DT餐厅),40.026988,116.459454', '肯德基(来广营餐厅),40.022285,116.471085', '汉堡王(北京广顺北大街餐厅),40.026723,116.472624', '肯德基(赛特奥莱餐厅),40.040734,116.513007', '鑫隆宫廷香酥牛肉饼,40.023231,116.473394', '吉野家(来广营欧尚店),40.022107,116.470012', '田老师红烧肉(望京诚盈中心店),40.026919,116.472583', '西山一面,40.05245,116.497096', '万世名流酒店CafeBerlin悦餐厅,40.033821,116.446783', '馋嘴米线(来广营店),40.028593,116.455671', '饭团长,40.043248,116.513136', '鑫鑫荷叶炒饭(欧尚上品店),40.022229,116.470231', '一碗汤面,40.023342,116.47304', '陕西老潼关肉夹馍,40.026928,116.472242', '金牌牛肉胡辣汤,40.04741,116.482071', '石磨面粉店,40.036495,116.450032', '老手艺熟食,40.047572,116.482004', '九九鸭脖王,40.039527,116.495714', '香河肉饼过桥米线,40.043759,116.473149', '知梨草堂轻奢新中式,40.030985,116.445999', '炸鸡汉堡,40.048188,116.482173', '川湘快餐,40.022522,116.508153', '烽洺快餐,40.023238,116.472918', '飞哥米线黄焖鸡,40.047476,116.446418', '维维豆奶,40.041753,116.47138', '庆丰包子铺(来广营店),40.026682,116.469663', '乔平麻辣烫,40.033721,116.481049', '张亮麻辣烫(马泉营店),40.04


现在爬取第 1 大区的第 5 区域
search complete
output:  [40.074925113333336, 116.37735956666667, 40.129114556666664, 116.44583843333334]
total:  205
['麦当劳(昌平定泗路餐厅),40.12323,116.420219', '肯德基(平西府餐厅),40.108784,116.389479', '肯德基(天通北苑餐厅),40.080842,116.420385', '田老师红烧肉(天通北苑店),40.080837,116.420915', '麦当劳(北七家餐厅),40.109122,116.379741', '肯德基(天通苑华联餐厅),40.07662,116.421882', '麦当劳(天通苑餐厅),40.081204,116.419123', '肯德基(北七家餐厅),40.123284,116.419264', '麦当劳(天通苑2餐厅),40.07967,116.442464', '吉野家(定泗路店),40.12326,116.420801', '地下小吃城,40.08108,116.417796', '田老师红烧肉(立汤路店),40.124125,116.418632', '老家肉饼(天通苑店),40.081011,116.44419', '德克士(温都水城餐厅),40.113199,116.378739', '华莱士(东三旗店),40.095447,116.424326', '吉野家(天通苑地铁站店),40.081208,116.419189', '热辣生活(北京华联天通苑购物中心店),40.076501,116.421102', '驴祥斋河间驴肉火烧,40.121451,116.439389', '老妈烫饭(北京瑞香原美食街店),40.112717,116.377465', '最高鸡密(温都水城店),40.108262,116.377919', '紫燕百味鸡(天通西苑店),40.080787,116.416028', '汉霸汉堡(天通苑店),40.078725,116.44222', '营养快餐(北七家物美店),40.123371,116.419389', '烤肉木桶饭,40.092652,116.420786', '主食厨房(太平星园

['肯德基(机场三餐厅),40.063228,116.621696', '麦当劳(首都机场餐厅),40.061051,116.624127', '汉堡王(北京首都国际机场店),40.06226,116.620892', '赛百味(机场店),40.054241,116.597549', '蜀国面庄(西平街),40.050491,116.598761', '凉皮先生(首都机场店),40.051362,116.598174', '好食汇(T3国际出港店),40.067384,116.597457', '伍氏猪脚(首都机场店),40.051245,116.601564', '重庆老麻抄手,40.035284,116.63484', '大鹅家快餐,40.032656,116.636141', '王小厨酸辣粉,40.051759,116.601459', '连杰快餐店,40.037471,116.638666', '嘉和一品(天竺店),40.054064,116.602043', '福森沙县小吃(机场店),40.054615,116.59814', '卤煮火烧(机场店),40.05041,116.598771', '杨国福麻辣烫(首都机场店),40.053917,116.601834', '湘云凌泉饺子馆(机场店),40.051111,116.598708', '侯师傅饺子城,40.036674,116.633985', '湘中滋味湖南米粉,40.051848,116.611245', '清真饺子馆(伊和轩店),40.050279,116.604286', '西安小吃(樱花园店),40.034409,116.634864', '金鼎小吃城,40.067715,116.591062', '小伍包子烧饼(机场店),40.051639,116.598591', '西安锋味小吃,40.046758,116.606352', '家香肉饼,40.059289,116.600917', '褡裢火烧,40.049233,116.598952', '张飞豆干廖记棒棒鸡,40.061171,116.620788', '张姐烤肉拌饭(机场店),40.054255,116.599956', '西域美食兰州老马拉面(顺义区李桥镇南半壁店村),40.038032,116.637595', '新大通熟食店,

['肯德基(顺义光明大街餐厅),40.131414,116.664001', '肯德基(马坡汽车穿梭餐厅),40.177578,116.658885', '麦当劳(顺义餐厅),40.132433,116.657156', '汉堡王(顺义华联餐厅),40.134795,116.659142', '肯德基(光明北大街餐厅),40.14841,116.663606', '仔仔烤鱼(俸伯店),40.138986,116.707222', '李先生(顺义店),40.134979,116.658223', '肯德基(顺义鑫海韵通餐厅),40.135439,116.658032', '汉堡王(北京隆华购物餐厅),40.132897,116.657206', '德克士(顺义金街餐厅),40.132533,116.65944', '肯德基(新顺餐厅),40.132385,116.658095', '田老师红烧肉(顺义店),40.141274,116.658015', '田老师红烧肉(绿港家园店),40.140022,116.671235', '肯德基(宜宾国泰餐厅),40.139007,116.653176', '和合谷(新顺南大街店),40.135126,116.657973', '和合谷(顺义双兴店),40.148567,116.663543', '周黑鸭(隆华购物中心店),40.133231,116.657178', '田老师红烧肉(马坡店),40.181467,116.658544', '味千拉面(新世界百货店),40.133195,116.658628', '成都美食(前俸伯西路),40.138663,116.705157', '李吉功夫煎饼(双兴鑫海韵通店),40.148568,116.66359', '老北京炸鸡,40.182571,116.654318', '正宗山东呛面馒头,40.138035,116.668722', '磨逗工坊(顺义店),40.134545,116.658871', '干哥俩风干牛肉干(顺义店),40.143446,116.660245', '过桥米线,40.140833,116.708254', '多克汉堡,40.135135,116.70928', '香南过桥米线,40.180769,116.65843', '奶茶嫁给粉(隆华店),40.13

In [5]:
# 9*9 区域搜索
if __name__ == '__main__':
    # ak
    #ak ="zmBxzD9D8mogOf8iQygQ5GTkMgKDgLT5"   # 这里填入你的百度API的ak
    ak = [
        "GsHF3EoEFsrRWv3L8Rd3shGHnBO8Zb45",
        "vOvll2Da1Uy78aHfiTgTAOZ5mZpFFO2f",
        "5UnM9eSMEDbfsjKg1IobksqHqGe8vjKB",
        "TXolGK5eEl3syP09a479uq4GbWgTCp3p",
        "AMQKttYPG1XgO7pSDbil8Lshswuvdnr6",
        "XywGfVo4kXI65eRr110G9r8ZdgNlTZsW",
        "MKrwWMxOir7Qc8ozfKmcUQusjPn6F0mB",
        "hv5PSMqxB8S01UMWrL6bFrj0e0Ow30PM",
        "RCaptGZviM4Ayj5gVMdjEM4dOzsFnIX4",
        "lfN9m3QcUNrIIyx7GrA3trSBxPomRmW4",
        "zmBxzD9D8mogOf8iQygQ5GTkMgKDgLT5",
        "Mep63CoO93mgoD3XlEFQ6MdTkNNugYKg"
    ]
    itemy = ["快餐"]        #不同的关键字，以 $ 隔开
    
    filename = ".\\area_segment.csv"       #  读取搜索区域
    f = open(filename,"rb")
    Lat_lng = pd.read_csv(f,header = None)
    f.close()
    
    start_time = time.time()
    print ("开始爬取数据，请稍等...", time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    
    num_global = 0
   
    for item in range(0, len(Lat_lng)):
        a = str(Lat_lng.iloc[item][0])+ "," + str(Lat_lng.iloc[item][1])+ ","+ str(Lat_lng.iloc[item][2])+","+str(Lat_lng.iloc[item][3])
        Area = LocaDiv(a)

        areas_to_use = Area.new_bounds()
        print(areas_to_use)
        i = 0
        num_single = 0
        for area_to_use in areas_to_use:
            i += 1
            print("现在爬取第" ,item+1 ,"大区的第",i, "区域")
            par = BaiDuPOI(area_to_use)  # 请修改爬取的类别
            [json_sel, total] = par.baidu_search()
            num_single += total
            par.write_poi(json_sel)
            
        print("第",item+1 ,"大区共爬取POI", num_single,"个")
        print("")
        num_global += num_single
      
    end_time = time.time()
    print ("爬取完毕，用时: ", float(end_time - start_time)/60, " 分钟",time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) )
    print("该项目共爬取POI",num_global,"个")
    print("")

In [24]:
http://api.map.baidu.com/place/v2/search?query=中学&region=昆明&output=json&ak="lCuBB2aY2Zh2k7jRslsGcq7dOmQgG6v5"

,0,1,2,3
0,30.379,114.1180,30.541,114.3915
1,30.379,114.3915,30.541,114.6650
2,30.541,114.1180,30.703,114.3915
3,30.541,114.3915,30.703,114.6650
